# Auto iBLR Gaussian Implementation Test 

In [1]:
using Random,LinearAlgebra, Flux.Optimise, Plots, ForneyLab
using ForneyLab:ParamStr2,renderCVI,naturalToBCParams,oneWindowSimulation_MCMC
import StatsBase:percentile,autocor

┌ Info: Precompiling ForneyLab [9fc3f58a-c2cc-5bff-9419-6a294fefdca9]
└ @ Base loading.jl:1278
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [2]:
Random.seed!(1234);

In [3]:
d=1
if d==2
    μ_in = [0.0,0.0]
    s_in = 100*[1 0;0 1.]
    λ_init = [s_in*μ_in;vec(-0.5*s_in)] # Natparams
    msg_in = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
else
    μ_in = 0.0
    s_in = 0.01
    λ_init = [s_in*μ_in,-0.5*s_in] # Natparams
    msg_in = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_in*μ_in,w=s_in)
end



Message: 𝒩(xi=0.00, w=1.00e-02)


In [4]:
#f(x) = x
# σ(x) = 1/(1+exp(-x))
# f(x)= [x^2+σ(x)tanh(x)+2*log(abs(x))*x,x]
f(x)= norm(x)

if d==2
    μ_out = [f(120.) .+ randn(),f(120.) .+ randn()]
    s_out = 1e12*[1 0;0 1.]
    msg_out = Message(Multivariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out)
else
    μ_out = f(120.) .+ randn()
    s_out = 100
    msg_out = Message(Univariate, GaussianWeightedMeanPrecision,xi=s_out*μ_out,w=s_out) 
end


logp_nc(z) = logPdf(msg_out.dist, f.(z))
println(msg_out.dist)

𝒩(xi=1.21e+04, w=100)



In [5]:
max_iterations = 1e3
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(eta=0.9,pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end
    


FE (734483.6207436061) is smaller now,FE_check starts when i=52
μ=-120.76399883674239,S=100.01
μ=-120.7346053039007,S=100.01
             VI result might be inaccurate!
μ=24.170996101413717,S=100.01000000000023


In [6]:
(typeof(msg_in.dist) <: Union{ProbabilityDistribution{Multivariate,F},ProbabilityDistribution{Univariate,F}} where F<:Gaussian)

true

In [7]:
max_iterations = 1e3
for str in ["free_energy","none","MonteCarlo"]
opt = ParamStr2(pareto_k_thr=0.05,max_iterations=max_iterations,mcmc_window_len=2000,
    rhat_cutoff=1.5,mcse_cutoff =1e-3,ess_threshold=150.,convergence_algo=str,verbose=true)
#opt= Descent(1e-7)
λ = renderCVI(logp_nc,Int64(max_iterations),opt,λ_init,msg_in)
λ_bc = naturalToBCParams(msg_in.dist,λ)
println("μ=$(λ_bc[1]),S=$(λ_bc[2])")
end

inexactLineSearch succeeded, setting initial stepsize to 0.14984999999999998
FE (6.35386552598513e9) is smaller now,FE_check starts when i=52
Algorithm converged at iteration 186
μ=120.86192033643914,S=100.01000000152749
inexactLineSearch succeeded, setting initial stepsize to 0.14984999999999998
μ=120.852993333987,S=100.01000000000005
inexactLineSearch succeeded, setting initial stepsize to 0.14984999999999998
             VI result might be inaccurate!
μ=0.0002959947031499155,S=100.01000000000023


In [8]:
# opts = ParamStr2(eta=0.1,pareto_k_thr=0.05,max_iterations=1e5,mcmc_window_len=1000,
#     rhat_cutoff=4,mcse_cutoff =1e-7,ess_threshold=150.,convergence_algo="MonteCarlo")
# opts.current_stepsize=opts.eta
# J=11
# Window=1000
# if d==2
#     η=[0.0, 0.01]
# λ_initials=[[randn(),randn()],1e12*[1 0;0 1.],1e1*[1 0;0 1.]]
# else
#     η=[0.0, 0.01]
#     λ_initials=[0.0, 0.01]
# end

# last_params,opt_matrix,stats_dict=oneWindowSimulation_MCMC(J,Window,opts,η,λ_initials,logp_nc,true,msg_in);
# stats_dict